In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [4]:

df = pd.read_csv("data/furth-data.csv")
print(df.info())
df.head()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40920 entries, 0 to 40919
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         40920 non-null  object 
 1   temperature  40920 non-null  float64
 2   humidity     40920 non-null  int64  
 3   rain         40920 non-null  float64
 4   pressure     40920 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.6+ MB
None


,time,temperature,humidity,rain,pressure
0,2020-01-01T00:00,-2.6,93,0.0,1037.0
1,2020-01-01T01:00,-2.7,93,0.0,1037.1
2,2020-01-01T02:00,-3.0,94,0.0,1036.7
3,2020-01-01T03:00,-3.6,94,0.0,1036.4
4,2020-01-01T04:00,-3.5,94,0.0,1036.4


In [5]:
X = df.drop(["rain", "time"], axis=1)
y = (df["rain"] > 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
best_model = None
best_score = 0

for i in range(10, 200, 20):
    print(f"Estimators: {i}")
    clf = RandomForestClassifier(n_estimators=i)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(clf.score(X_test, y_test))
    if score > best_score:
        best_model = clf

Estimators: 10
0.8207478005865103
Estimators: 30
0.8258797653958945
Estimators: 50
0.8289345063538612
Estimators: 70
0.8263685239491692
Estimators: 90
0.8299120234604106
Estimators: 110
0.8288123167155426
Estimators: 130
0.8288123167155426
Estimators: 150
0.8268572825024438
Estimators: 170
0.8290566959921799
Estimators: 190
0.8315004887585533


In [7]:
X_train[:1]

,temperature,humidity,pressure
5129,31.9,29,1011.6


In [15]:
from skl2onnx import to_onnx
import numpy as np 

input = np.array(X_train[:1]).astype(np.float32)
print(input.shape)

onx = to_onnx(best_model, input, target_opset=12)
with open("rain_predictor_forest.onnx", "wb") as f:
    f.write(onx.SerializeToString())

(1, 3)
